In [287]:
#import mobi
#import tempfile
#import shutil
#from extract import extract
#tempdir, filepaths = extract(r"C:\Users\Mateusz\Downloads\Steinbeck John - Grona gniewu.mobi")
#print(filepaths)
#shutil.rmtree(tempdir)

In [288]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd
from datetime import datetime
from opensearchpy import OpenSearch, helpers
from dateutil.relativedelta import relativedelta

In [289]:
df = pd.read_csv("europe_cities.csv")

In [290]:
#df = df[(df['lat']>36) & (df['lng']>-10) & (df['lng'] < 50) & (df['population'] > 350000 ) ]
#df = df.sort_values("id")
#df.to_csv("europe_cities.csv")

In [291]:
lats = df["lat"].to_list()
lngs = df['lng'].to_list()
ids = df.loc[:, ['id','city','country']].values.tolist()

In [292]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": lats,
	"longitude": lngs,
	"current": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m", "cloud_cover", "surface_pressure"],
	"timezone": "Europe/Berlin",
    
}
responses = openmeteo.weather_api(url, params=params)

In [293]:
temp = dict()
result = list()

In [294]:
for id,response,lat,lng in zip(ids,responses,lats, lngs):

    current = response.Current()
    temp = dict()
    temp_datetime_id = int(datetime.fromtimestamp(current.Time()).strftime('%Y%m%d%H%M%S'))
    temp["city"] = id[1]
    temp["country"] = id[2]
    temp['location'] = [lng, lat]
    temp["datetime_id"] = temp_datetime_id
    temp["temperature"] = current.Variables(0).Value()
    temp["humidity"] = current.Variables(1).Value()
    temp["wind_speed"] =  current.Variables(2).Value()
    temp["wind_direction"] = current.Variables(3).Value()
    temp["wind_gusts"] = current.Variables(4).Value()
    temp["cloud_cover"] = current.Variables(5).Value()
    temp["surface_pressure"] = current.Variables(6).Value()
    result.append( (f"{str(id[0])}_{str(temp_datetime_id)}", temp) ) #tuple(id, dict)


In [295]:
for res in result:
    if res[1]['city'] == 'Łódź':
        print(res[1])

{'city': 'Łódź', 'country': 'Poland', 'location': [19.4547, 51.7769], 'datetime_id': 20251129174500, 'temperature': -0.3499999940395355, 'humidity': 93.0, 'wind_speed': 3.9600000381469727, 'wind_direction': 165.0, 'wind_gusts': 12.239999771118164, 'cloud_cover': 100.0, 'surface_pressure': 989.4152221679688}


In [296]:
host = 'localhost'
port = 9200

# Create the client with SSL/TLS and hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    http_auth=('admin', 'QWERTYadmin123!@#'),  # login/admin password
)

In [297]:
""" response = client.indices.delete(
    index = 'python-weather5-index'
) """

" response = client.indices.delete(\n    index = 'python-weather5-index'\n) "

In [298]:
# Create an index with non-default settings.
index_name = 'python-weather5-index'
mapping = {
    "mappings": {
        "properties": {
            "city": {"type": "text"},
            "country": {"type": "keyword"},
            "location": {"type": "geo_point"},  # lat/lon for maps
            "datetime_id": {"type": "date", "format": "yyyyMMddHHmmss"},
            "temperature": {"type": "float"},
            "humidity": {"type": "float"},
            "wind_speed": {"type": "float"},
            "wind_direction": {"type": "float"},
            "wind_gusts": {"type": "float"},
            "cloud_cover": {"type": "float"},
            "surface_pressure": {"type": "float"}
        }
    }
}
response = client.indices.create(index=index_name, body=mapping)
print('\nCreating index:')
print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-weather5-index'}


In [299]:
actions = [
    {
        '_op_type': 'update',
        '_index': 'python-weather5-index',
        '_id': res[0],                    # document id
        'doc': res[1],
        'doc_as_upsert': True          # if doc doesn't exist -> create
    } for res in result
]

In [300]:
success, failed = helpers.bulk(client, actions, refresh=True)
print(f"Success: {success}, Failed: {failed}")

Success: 224, Failed: []
